# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [ ]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

In [1]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [2]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [3]:
from deep.constants import *
from deep.utils import *

In [4]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [7]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)


tot_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_train_val.csv')).drop_duplicates()

test_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_test_v0.7.1.csv'))

In [9]:
minimal_results = {
    'present_prim_tags': 0.8,
    'present_sec_tags': 0.85,
    'sectors': 0.89,
    'pillars_1d': 0.85,
    'pillars_2d': 0.81,
    'subpillars_2d_part1': 0.8, 
   'subpillars_2d_part2': 0.72,
   'subpillars_1d_part1': 0.89,
   'subpillars_1d_part2': 0.85,
   'subpillars_1d_part3': 0.84,
    'gender': 0.84,
    'age': 0.84,
    'specific_needs_groups': 0.79,
    'affected_groups_levels_2_3': 0.99,
    'gender_snorkel': 0.87
}

In [19]:
columns = ['excerpt', 'entry_id',
           'present_prim_tags',
           #'sectors',
           'pillars_2d',
           #'pillars_1d',
           'subpillars_2d_part1', 
           'subpillars_2d_part2'
           #'subpillars_1d_part1',
           #'subpillars_1d_part2',
           #'subpillars_1d_part3'
        ]

In [ ]:
columns = [
    'excerpt', 'entry_id',
    'present_sec_tags',
    'gender',
    'age',
    'specific_needs_groups',
    'affected_groups_levels_2_3',
    'gender_snorkel'
]

In [ ]:
columns = [
    'excerpt', 'entry_id',
    'present_sec_tags',
    'gender',
    'age',
    'specific_needs_groups',
    'gender_snorkel'
]

In [22]:

tot_df = tot_df[columns]
test_df = test_df[columns]

In [23]:
tot_df

,excerpt,entry_id,present_prim_tags,pillars_2d,subpillars_2d_part1,subpillars_2d_part2
0,"After past, partially implemented attempts, th...",489435,"['sectors', 'subpillars_2d', 'subpillars_1d']",['Capacities & Response'],['Capacities & Response->International Response'],['Capacities & Response->National Response']
1,"[10th November, NW Syria] Now with the lockdow...",194719,['subpillars_1d'],[],[],[]
2,El Salvador hace frente a una combinación de c...,186152,"['sectors', 'subpillars_2d']",['Humanitarian Conditions'],['Humanitarian Conditions->Living Standards'],[]
3,Extreme poverty and the government’s fiscal li...,489431,"['sectors', 'subpillars_2d', 'subpillars_1d']",['Humanitarian Conditions'],['Humanitarian Conditions->Living Standards'],[]
4,Las personas que habitan en cantones donde no ...,186986,['sectors'],[],[],[]
...,...,...,...,...,...,...
422322,"En 2021, UNICEF tiene acciones priorizadas en ...",500721,[],[],[],[]
422323,29 percent of women and 24 percent of men who ...,500108,[],[],[],[]
422324,29% des femmes et 24% des hommes qui ont accès...,500108,[],[],[],[]
422325,The emergency generated by the Covid-19 pandem...,500645,[],[],[],[]


## Sagemaker Prep

### Session

Configure SageMaker

In [24]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

In [25]:
SAGEMAKER_ROLE

'AmazonSageMaker-ExecutionRole-20210519T102514'

### Bucket upload

You need to upload data to an S3 bucket. 




In [26]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [30]:
DEV_BUCKET

S3Path('s3://sagemaker-deep-experiments-dev')

In [29]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=10000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

In [31]:
job_name

'pytorch-2021-11-29-14-46-21-478-all-models'

### Estimator Definition

In [32]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [ ]:
from sagemaker.pytorch import PyTorch

instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-all-models-experiments",
    'max_len': 512,
    'epochs': 6,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'output_length': 256,
    'training_names':','.join(columns[2:]),
    "instance_type": instance_type,
    'beta_f1': 0.5,
    'nb_repetitions': 2,
    'run_name': 'models_primary_tags',
    'min_results': str(minimal_results)
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/nicolo/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [ ]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [ ]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)